In [18]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.{DataTypes, StructField}
import org.apache.spark.sql.types._

import java.util.Date
import java.time.format.DateTimeFormatter
import org.apache.spark.sql.{functions => F}
import org.apache.spark.sql.{functions => T}
import org.apache.spark.sql.expressions
import org.apache.spark.sql.expressions.Window
//import spark.implicits._

Unknown Error: <console>:45: error: stable identifier required, but spark.implicits found.
       import spark.implicits._
                    ^


In [19]:
// создаем spark сессию
val spark:SparkSession = SparkSession.builder()
      .master("local[]")
      .appName("Task4")
      .getOrCreate()

spark = org.apache.spark.sql.SparkSession@62294e78


org.apache.spark.sql.SparkSession@62294e78

In [20]:
 // Даты, на которые будет строиться витрина
val pDate1 = "2020-11-01"
val pDate2 = "2020-11-02"
val pDate3 = "2020-11-03"
val pDate4 = "2020-11-04"

// Общий параметр для запуска витрин
// Для запуска витрин просто пишем: val pDateCommon = pDate1 или val pDateCommon = pDate2 и.т.д

val pDateCommon = pDate1 // Все витрины будут построены на дату pDate2

pDate1 = 2020-11-01
pDate2 = 2020-11-02
pDate3 = 2020-11-03
pDate4 = 2020-11-04
pDateCommon = 2020-11-01


2020-11-01

In [21]:
// Определим схему данных для таблицы клиентов
val schemaClients = DataTypes.createStructType(Array[StructField](
      DataTypes.createStructField("ClientId", DataTypes.IntegerType, true),
      DataTypes.createStructField("ClientName", DataTypes.StringType, true),
      DataTypes.createStructField("Type", DataTypes.StringType, true),
      DataTypes.createStructField("Form", DataTypes.StringType, true),  
      DataTypes.createStructField("RegisterDate", DataTypes.DateType, true)))

// Определим схему данных для таблицы счетов
val schemaAccounts = StructType(Array[StructField](
      DataTypes.createStructField("AccountId", DataTypes.IntegerType, false),
      DataTypes.createStructField("AccountNum", DataTypes.StringType, false),
      DataTypes.createStructField("ClientId", DataTypes.IntegerType, false),
      DataTypes.createStructField("DateOpen", DataTypes.DateType, false)))

// Определим схему данных для таблицы банковских операций 
val schemaOperations = StructType(Array[StructField](
      DataTypes.createStructField("AccountDB", DataTypes.IntegerType, true),
      DataTypes.createStructField("AccountCR", DataTypes.IntegerType, true),
      DataTypes.createStructField("DateOp", DataTypes.DateType, true),
      DataTypes.createStructField("Amount", DecimalType(10, 2), false),
      DataTypes.createStructField("Currency", DataTypes.StringType, true), 
      DataTypes.createStructField("Comment", StringType, true)))                                          
// Определим схему данных для таблицы курсов валют
val schemaRates = DataTypes.createStructType(Array[StructField](
      DataTypes.createStructField("Currency", DataTypes.StringType, false),
      DataTypes.createStructField("Rate", DecimalType(10, 2), false),
      DataTypes.createStructField("RateDate", DataTypes.DateType, false)))

schemaClients = StructType(StructField(ClientId,IntegerType,true),StructField(ClientName,StringType,true),StructField(Type,StringType,true),StructField(Form,StringType,true),StructField(RegisterDate,DateType,true))
schemaAccounts = StructType(StructField(AccountId,IntegerType,false),StructField(AccountNum,StringType,false),StructField(ClientId,IntegerType,false),StructField(DateOpen,DateType,false))
schemaOperations = StructType(StructField(AccountDB,IntegerType,true),StructField(AccountCR,IntegerType,true),StructField(DateOp,DateType,true),StructField(Amount,DecimalType(10,2),false),StructField(Currency,StringType,true),StructField(Comment,StringType,true))


schemaRates: ...


StructType(StructField(AccountDB,IntegerType,true),StructField(AccountCR,IntegerType,true),StructField(DateOp,DateType,true),StructField(Amount,DecimalType(10,2),false),StructField(Currency,StringType,true),StructField(Comment,StringType,true))

In [22]:
// чтение полного набора данных
// считаем csv файл клиентов Clients.csv, запишем прочитанное в dataframe
val dfClients = spark.read.format("csv")
        .option("header", "true")
        //.option("encoding", "KOI8_U")
        .option("multiline", true)
        .option("sep", ";")
        .option("dateformat","yyyy-MM-dd")
        .option("enforceSchema", true)
        .schema(schemaClients)
        .load("files/Clients.csv")
dfClients.show()

// считаем csv файл счетов Account.csv, запишем прочитанное в dataframe
val dfAccounts = spark.read.format("csv")
        .option("header", "true")
        .option("multiline", true)
        .option("sep", ";")
        .option("dateformat","yyyy-MM-dd")
        .option("enforceSchema", true)
        .schema(schemaAccounts)
        .load("files/Account.csv")
dfAccounts.show()

// считаем csv файл банковских операций Operation.csv, запишем прочитанное в dataframe
val dfOperations = spark.read.format("csv")
        .option("header", "true")
        .option("multiline", true)
        .option("sep", ";")
        .option("enforceSchema", true)
        .option("dateformat","yyyy-MM-dd")
        .schema(schemaOperations)
        .load("files/Operation.csv")
dfOperations.show()

// считаем csv файл курса валют Rate.csv, запишем прочитанное в dataframe
val dfRates = spark.read.format("csv")
        .option("header", "true")
        .option("multiline", true)
        .option("sep", ";")
        .option("enforceSchema", true)
        .schema(schemaRates)
        .load("files/Rate.csv")
dfRates.show()

+--------+----------+----+----+------------+
|ClientId|ClientName|Type|Form|RegisterDate|
+--------+----------+----+----+------------+
|       1|  Клиент 1|   Ф|   -|  2020-11-01|
|       2|  Клиент 2|   Ю| ООО|  2020-11-02|
|       3|  Клиент 3|   Ф|   -|  2020-11-03|
|       4|  Клиент 4|   Ю|  ИП|  2020-11-04|
|       5|  Клиент 5|   Ф|   -|  2020-11-01|
|       6|  Клиент 6|   Ю|  АО|  2020-11-02|
|       7|  Клиент 7|   Ф|   -|  2020-11-03|
|       8|  Клиент 8|   Ю| ПАО|  2020-11-01|
|       9|  Клиент 9|   Ф|   -|  2020-11-02|
|      10| Клиент 10|   Ю| ООО|  2020-11-03|
|      11| Клиент 11|   Ф|   -|  2020-11-04|
|      12| Клиент 12|   Ю|  ИП|  2020-11-01|
|      13| Клиент 13|   Ф|   -|  2020-11-02|
|      14| Клиент 14|   Ю|  АО|  2020-11-03|
|      15| Клиент 15|   Ф|   -|  2020-11-01|
|      16| Клиент 16|   Ю| ПАО|  2020-11-02|
|      17| Клиент 17|   Ф|   -|  2020-11-03|
|      18| Клиент 18|   Ю| ООО|  2020-11-04|
|      19| Клиент 19|   Ф|   -|  2020-11-01|
|      20|

dfClients = [ClientId: int, ClientName: string ... 3 more fields]
dfAccounts = [AccountId: int, AccountNum: string ... 2 more fields]
dfOperations = [AccountDB: int, AccountCR: int ... 4 more fields]
dfRates = [Currency: string, Rate: decimal(10,2) ... 1 more field]


[Currency: string, Rate: decimal(10,2) ... 1 more field]

In [42]:
/// Считаем строки из БД postgres
// СУБД postgres находится локально

val lists = spark.read
    .format("jdbc")
    .option("url", "jdbc:postgresql://192.168.84.137:5435/bank")
    .option("dbtable", "list")
    .option("user", "postgres")
    .option("password", "postgres")
    .option("driver", "org.postgresql.Driver")
    .load()
lists.select("maskvalue").filter("Id = 1").show()

lists = [id: int, maskvalue: string]


+---------+
|maskvalue|
+---------+
|        %|
+---------+



[id: int, maskvalue: string]

In [29]:
val clientsAndAccounts = dfClients.join(dfAccounts,"ClientId")

clientsAndAccounts = [ClientId: int, ClientName: string ... 6 more fields]


[ClientId: int, ClientName: string ... 6 more fields]

In [30]:
clientsAndAccounts.orderBy("ClientId").count()

20000

In [31]:
clientsAndAccounts.show()

+--------+----------+----+----+------------+---------+--------------------+----------+
|ClientId|ClientName|Type|Form|RegisterDate|AccountId|          AccountNum|  DateOpen|
+--------+----------+----+----+------------+---------+--------------------+----------+
|       1|  Клиент 1|   Ф|   -|  2020-11-01|        1|40702810927050000337|2020-11-01|
|       2|  Клиент 2|   Ю| ООО|  2020-11-02|        2|40802810300000009067|2020-11-02|
|       3|  Клиент 3|   Ф|   -|  2020-11-03|        3|40802810300000009708|2020-11-03|
|       4|  Клиент 4|   Ю|  ИП|  2020-11-04|        4|40802810800000030701|2020-11-04|
|       5|  Клиент 5|   Ф|   -|  2020-11-01|        5|40802810300000011071|2020-11-01|
|       6|  Клиент 6|   Ю|  АО|  2020-11-02|        6|40802810100000063339|2020-11-02|
|       7|  Клиент 7|   Ф|   -|  2020-11-03|        7|40702810823620000031|2020-11-03|
|       8|  Клиент 8|   Ю| ПАО|  2020-11-01|        8|40802810409260005894|2020-11-04|
|       9|  Клиент 9|   Ф|   -|  2020-11-02

In [32]:
//Делаем join таблицы ClientsAndAccounts и Operations
val clientsAndAccountsOp = dfOperations
    .join(clientsAndAccounts.select($"ClientId".as("ClientDB"),
                                    $"AccountId".as("AccDB"),
                                    $"AccountNum".as("NumDB"),
                                    $"Type".as("TypeDB"),
                                    $"Form".as("FormDB")
                                   ),
          $"AccDB"===$"AccountDB" || $"AccDB" === $"AccountCR","left")
    .join(clientsAndAccounts.select($"ClientId".as("ClientCR"),
                                    $"AccountId".as("AccCR"),
                                    $"AccountNum".as("NumCR"),
                                    $"Type".as("TypeCR"),
                                    $"Form".as("FormCR")
                                   ),
         $"AccCR"===$"AccountCR","left")

clientsAndAccountsOp = [AccountDB: int, AccountCR: int ... 14 more fields]


[AccountDB: int, AccountCR: int ... 14 more fields]

In [33]:
clientsAndAccountsOp.show()

+---------+---------+----------+------+--------+--------------------+--------+-----+--------------------+------+------+--------+-----+--------------------+------+------+
|AccountDB|AccountCR|    DateOp|Amount|Currency|             Comment|ClientDB|AccDB|               NumDB|TypeDB|FormDB|ClientCR|AccCR|               NumCR|TypeCR|FormCR|
+---------+---------+----------+------+--------+--------------------+--------+-----+--------------------+------+------+--------+-----+--------------------+------+------+
|        8|     5006|2020-11-04|  8.00|     USD|турусл, турист, з...|       8|    8|40802810409260005894|     Ю|   ПАО|    5006| 5006|40702810020270000072|     Ю|    АО|
|        8|     5006|2020-11-04|  8.00|     USD|турусл, турист, з...|    5006| 5006|40702810020270000072|     Ю|    АО|    5006| 5006|40702810020270000072|     Ю|    АО|
|        5|     5003|2020-11-01|  5.00|     USD|компью, монитор, ...|       5|    5|40802810300000011071|     Ф|     -|    5003| 5003|4080281052027000

In [34]:
// cформируем временную таблицу для sql
dfRates.createOrReplaceTempView("rates")
// определим курс на подследнюю дату
var lastRate = spark.sql(s"""SELECT Currency, Rate
                             FROM rates
                             WHERE RateDate = (SELECT MAX(RateDate) FROM rates)  
                       """)

import org.apache.spark.sql.functions._

// зададим условие для объединения
val rateCondition = clientsAndAccountsOp.col("Currency") === lastRate.col("Currency")
// join
var clientsAndAccountsOpRate = clientsAndAccountsOp
                .join(lastRate,rateCondition,"left")
                .drop(lastRate.col("Currency"))

// cформируем результирующую таблицу, с приведением рублевых сумм
clientsAndAccountsOpRate = clientsAndAccountsOpRate
        .withColumn("AmountRUB",round(col("Amount")*col("Rate"),2))
        .where($"DateOp" === pDateCommon)
                                
clientsAndAccountsOpRate.show()

java.lang.AssertionError: assertion failed: 
  Decimal$DecimalIsFractional
     while compiling: <console>
        during phase: globalPhase=terminal, enteringPhase=jvm
     library version: version 2.12.15
    compiler version: version 2.12.15
  reconstructed args: -usejavacp -classpath file:/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/ext/cldrdata.jar:file:/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/ext/dnsns.jar:file:/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/ext/icedtea-sound.jar:file:/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/ext/localedata.jar:file:/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/ext/sunjce_provider.jar:file:/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/ext/sunec.jar:file:/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/ext/nashorn.jar:file:/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/ext/jaccess.jar:file:/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/ext/sunpkcs11.jar:file:/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/ext/zipfs.jar:file:/opt/spark-3.3.1-bin-hadoop3/conf/:file:/opt/spark-3.3.1-bin-h

	at org.codehaus.janino.UnitCompiler.getType(UnitCompiler.java:6490)
	at org.codehaus.janino.UnitCompiler.findMostSpecificIInvocable(UnitCompiler.java:9306)
	at org.codehaus.janino.UnitCompiler.findIMethod(UnitCompiler.java:9192)
	at org.codehaus.janino.UnitCompiler.findIMethod(UnitCompiler.java:9110)
	at org.codehaus.janino.UnitCompiler.compileGet2(UnitCompiler.java:5055)
	at org.codehaus.janino.UnitCompiler.access$9100(UnitCompiler.java:226)
	at org.codehaus.janino.UnitCompiler$16.visitMethodInvocation(UnitCompiler.java:4482)
	at org.codehaus.janino.UnitCompiler$16.visitMethodInvocation(UnitCompiler.java:4455)
	at org.codehaus.janino.Java$MethodInvocation.accept(Java.java:5286)
	at org.codehaus.janino.UnitCompiler.compileGet(UnitCompiler.java:4455)
	at org.codehaus.janino.UnitCompiler.compileGetValue(UnitCompiler.java:5683)
	at org.codehaus.janino.UnitCompiler.compile2(UnitCompiler.java:3839)
	at org.codehaus.janino.UnitCompiler.access$6100(UnitCompiler.java:226)
	at org.codehaus.jan

	at org.codehaus.janino.Java$Block.accept(Java.java:2969)
	at org.codehaus.janino.UnitCompiler.compile(UnitCompiler.java:1490)
	at org.codehaus.janino.UnitCompiler.compile2(UnitCompiler.java:1848)
	at org.codehaus.janino.UnitCompiler.access$2200(UnitCompiler.java:226)
	at org.codehaus.janino.UnitCompiler$6.visitWhileStatement(UnitCompiler.java:1501)
	at org.codehaus.janino.UnitCompiler$6.visitWhileStatement(UnitCompiler.java:1490)
	at org.codehaus.janino.Java$WhileStatement.accept(Java.java:3245)
	at org.codehaus.janino.UnitCompiler.compile(UnitCompiler.java:1490)
	at org.codehaus.janino.UnitCompiler.compileStatements(UnitCompiler.java:1573)
	at org.codehaus.janino.UnitCompiler.compile(UnitCompiler.java:3420)
	at org.codehaus.janino.UnitCompiler.compileDeclaredMethods(UnitCompiler.java:1362)
	at org.codehaus.janino.UnitCompiler.compileDeclaredMethods(UnitCompiler.java:1335)
	at org.codehaus.janino.UnitCompiler.compile2(UnitCompiler.java:807)
	at org.codehaus.janino.UnitCompiler.compile

+---------+---------+----------+------+--------+--------------------+--------+-----+--------------------+------+------+--------+-----+--------------------+------+------+-----+---------+
|AccountDB|AccountCR|    DateOp|Amount|Currency|             Comment|ClientDB|AccDB|               NumDB|TypeDB|FormDB|ClientCR|AccCR|               NumCR|TypeCR|FormCR| Rate|AmountRUB|
+---------+---------+----------+------+--------+--------------------+--------+-----+--------------------+------+------+--------+-----+--------------------+------+------+-----+---------+
|        1|     4999|2020-11-01|  1.00|     RUB|а/м, а\м, автомоб...|       1|    1|40702810927050000337|     Ф|     -|    4999| 4999|40802810147180000076|     Ф|     -| 1.00|     1.00|
|        1|     4999|2020-11-01|  1.00|     RUB|а/м, а\м, автомоб...|    4999| 4999|40802810147180000076|     Ф|     -|    4999| 4999|40802810147180000076|     Ф|     -| 1.00|     1.00|
|        5|     5003|2020-11-01|  5.00|     USD|компью, монитор, ...| 

lastRate = [Currency: string, Rate: decimal(10,2)]
rateCondition = (Currency = Currency)
clientsAndAccountsOpRate = [AccountDB: int, AccountCR: int ... 16 more fields]
clientsAndAccountsOpRate = [AccountDB: int, AccountCR: int ... 16 more fields]


error: error while loading Decimal, class file '/opt/spark-3.3.1-bin-hadoop3/jars/spark-catalyst_2.12-3.3.1.jar(org/apache/spark/sql/types/Decimal.class)' is broken
(class java.lang.RuntimeException/error reading Scala signature of Decimal.class: assertion failed:
  Decimal$DecimalIsFractional
     while compiling: <console>
        during phase: globalPhase=terminal, enteringPhase=jvm
     library version: version 2.12.15
    compiler version: version 2.12.15
  reconstructed args: -usejavacp -classpath file:/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/ext/cldrdata.jar:file:/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/ext/dnsns.jar:file:/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/ext/icedtea-sound.jar:file:/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/ext/localedata.jar:file:/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/ext/sunjce_provider.jar:file:/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/ext/sunec.jar:file:/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/ext/nashorn.jar:file:/usr/lib/jvm/java-8-openjdk-amd

[AccountDB: int, AccountCR: int ... 16 more fields]

In [35]:
val corporate_payments = clientsAndAccountsOpRate.groupBy($"AccDB".as("AccountID"), 
                                                          $"ClientDB".as("ClientId"), 
                                                          $"DateOp".as("CutoffDt"))
        .agg(round(sum(when($"AccDB" === $"AccountDB", $"AmountRub").otherwise(0)), 2).as("PaymentAmt"),
             round(sum(when($"AccDB" === $"AccountCR", $"AmountRub").otherwise(0)), 2).as("EnrollementAmt"),
             round(sum(when($"AccDB" === $"AccountDB" && $"NumCR".startsWith("40702"),$"AmountRub").otherwise(0)), 2).as("TaxAmt"),
             round(sum(when($"AccDB" =

Syntax Error.: 

In [36]:
//Сумма операций по счету, где счет клиента указан в дебете проводки
clientsAndAccountsOpRate.groupBy($"AccountDB",$"ClientDB",$"DateOp").agg(sum($"AmountRub").as("PaymentAmt")).show()

+---------+--------+----------+-----------+
|AccountDB|ClientDB|    DateOp| PaymentAmt|
+---------+--------+----------+-----------+
|    11581|    1580|2020-11-01|12956099.37|
|    13705|    3704|2020-11-01|13429262.85|
|    10581|     580|2020-11-01|14830014.00|
|    16461|    6461|2020-11-01|16558734.00|
|    15153|    5152|2020-11-01|16174182.00|
|    17189|    7188|2020-11-01|11198727.97|
|    18981|    8981|2020-11-01|17299614.00|
|    16041|    6040|2020-11-01|16435254.00|
|    13457|    3456|2020-11-01|10374963.61|
|     6493|    6493|2020-11-01|11822645.61|
|    11825|    1824|2020-11-01|10014732.25|
|     6485|    1483|2020-11-01| 8836034.05|
|    12057|    2056|2020-11-01|15263958.00|
|    15969|    5969|2020-11-01|16414086.00|
|     9357|    9357|2020-11-01|14470158.00|
|     8781|    8781|2020-11-01|14300814.00|
|    10573|     572|2020-11-01|12731547.21|
|    12197|    2196|2020-11-01|10096843.81|
|      985|    5983|2020-11-01|10595628.45|
|     7157|    7157|2020-11-01| 

In [37]:
clientsAndAccountsOpRate.groupBy($"AccDB".as("AccountID"), $"ClientDB".as("ClientId"), $"DateOp")   
        .agg(sum(when($"AccDB" === $"AccountDB", $"AmountRub")).as("PaymentAmt"),
             sum(when($"AccDB" === $"AccountCR", $"AmountRub")).as("EnrollementAmt")
            ).show()

+---------+--------+----------+-----------+--------------+
|AccountID|ClientId|    DateOp| PaymentAmt|EnrollementAmt|
+---------+--------+----------+-----------+--------------+
|     5167|    5167|2020-11-01|       null|   10413848.13|
|     5375|    5375|2020-11-01|       null|    7487815.21|
|      557|     557|2020-11-01| 7527546.61|          null|
|     5835|    5835|2020-11-01|       null|   11965278.00|
|     1201|    1201|2020-11-01|10643746.77|          null|
|     6563|    6563|2020-11-01|       null|    7750042.45|
|     7163|    7163|2020-11-01|       null|    7882480.45|
|     2605|    2605|2020-11-01|10956515.85|          null|
|     8471|    8471|2020-11-01|       null|    8171195.29|
|     8787|    8787|2020-11-01|       null|   12833166.00|
|     8791|    8791|2020-11-01|       null|   11221166.61|
|     3877|    3877|2020-11-01|11239879.29|          null|
|     4133|    4133|2020-11-01| 8316877.09|          null|
|     4181|    4181|2020-11-01| 8327472.13|          nul

In [38]:
//построение TaxAmt
//Сумму операций, где счет клиента указан в дебете, и счет кредита 40702
 clientsAndAccountsOpRate.groupBy($"AccDB".as("AccountID"), $"ClientDB".as("ClientId"), $"DateOp")   
        .agg(round(sum(when($"AccDB" === $"AccountDB", $"AmountRub").otherwise(0)), 2).as("PaymentAmt"),
             round(sum(when($"AccDB" === $"AccountCR", $"AmountRub").otherwise(0)), 2).as("EnrollementAmt"),
             round(sum(when($"AccDB" === $"AccountDB" && $"NumCR".startsWith("40702"),$"AmountRub").otherwise(0)), 2).as("TaxAmt")
            ).show()

+---------+--------+----------+-----------+--------------+-----------+
|AccountID|ClientId|    DateOp| PaymentAmt|EnrollementAmt|     TaxAmt|
+---------+--------+----------+-----------+--------------+-----------+
|     8592|    8592|2020-11-01|       0.00|   11508632.89|       0.00|
|     8932|    8932|2020-11-01|       0.00|   17285502.00|       0.00|
|    12559|    2559|2020-11-01|       0.00|   12060563.97|       0.00|
|     5375|    5375|2020-11-01|       0.00|    7487815.21|       0.00|
|    16461|    6461|2020-11-01|16558734.00|          0.00|       0.00|
|     1752|    1752|2020-11-01|       0.00|    9998839.69|       0.00|
|    18981|    8981|2020-11-01|17299614.00|          0.00|       0.00|
|     6493|    6493|2020-11-01|11822645.61|          0.00|       0.00|
|     4784|    4784|2020-11-01|       0.00|   13669854.45|       0.00|
|    15969|    5969|2020-11-01|16414086.00|          0.00|16414086.00|
|     6563|    6563|2020-11-01|       0.00|    7750042.45|       0.00|
|     

In [39]:
//построение ClearAmt
//Сумма операций, где счет клиента указан в кредите, и счет дебета 40802
clientsAndAccountsOpRate.groupBy($"AccDB".as("AccountID"), $"ClientDB".as("ClientId"), $"DateOp")   
        .agg(round(sum(when($"AccDB" === $"AccountDB", $"AmountRub").otherwise(0)), 2).as("PaymentAmt"),
             round(sum(when($"AccDB" === $"AccountCR", $"AmountRub").otherwise(0)), 2).as("EnrollementAmt"),
             round(sum(when($"AccDB" === $"AccountDB" && $"NumCR".startsWith("40702"),$"AmountRub").otherwise(0)), 2).as("TaxAmt"),
             round(sum(when($"AccDB" === $"AccountCR" && $"NumDB".startsWith("40802"),$"AmountRub").otherwise(0)), 2).as("ClearAmt")
            ).show()

+---------+--------+----------+-----------+--------------+-----------+-----------+
|AccountID|ClientId|    DateOp| PaymentAmt|EnrollementAmt|     TaxAmt|   ClearAmt|
+---------+--------+----------+-----------+--------------+-----------+-----------+
|     5167|    5167|2020-11-01|       0.00|   10413848.13|       0.00|       0.00|
|     5375|    5375|2020-11-01|       0.00|    7487815.21|       0.00| 7487815.21|
|      557|     557|2020-11-01| 7527546.61|          0.00| 7527546.61|       0.00|
|     5835|    5835|2020-11-01|       0.00|   11965278.00|       0.00|11965278.00|
|     1201|    1201|2020-11-01|10643746.77|          0.00|       0.00|       0.00|
|     6563|    6563|2020-11-01|       0.00|    7750042.45|       0.00| 7750042.45|
|     7163|    7163|2020-11-01|       0.00|    7882480.45|       0.00| 7882480.45|
|     2605|    2605|2020-11-01|10956515.85|          0.00|10956515.85|       0.00|
|     8471|    8471|2020-11-01|       0.00|    8171195.29|       0.00|       0.00|
|   

In [50]:
//построение CarsAmt
//Сумма операций, где счет клиента указан в дебете проводки и назначение платежа не содержит слов по маскам Списка 1
clientsAndAccountsOpRate.groupBy($"AccDB".as("AccountID"), $"ClientDB".as("ClientId"), $"DateOp".as("CutoffDt"))   
        .agg(round(sum(when($"AccDB" === $"AccountDB", $"AmountRub").otherwise(0)), 2).as("PaymentAmt"),
             round(sum(when($"AccDB" === $"AccountCR", $"AmountRub").otherwise(0)), 2).as("EnrollementAmt"),
             round(sum(when($"AccDB" === $"AccountDB" && $"NumCR".startsWith("40702"),$"AmountRub").otherwise(0)), 2).as("TaxAmt"),
             round(sum(when($"AccDB" === $"AccountCR" && $"NumDB".startsWith("40802"),$"AmountRub").otherwise(0)), 2).as("ClearAmt"),
             round(sum(when($"AccDB" === $"AccountDB", $"AmountRub").otherwise(0)), 2).as("CarsAmt")
            ).show()

+---------+--------+----------+-----------+--------------+-----------+-----------+-----------+
|AccountID|ClientId|  CutoffDt| PaymentAmt|EnrollementAmt|     TaxAmt|   ClearAmt|    CarsAmt|
+---------+--------+----------+-----------+--------------+-----------+-----------+-----------+
|     8592|    8592|2020-11-01|       0.00|   11508632.89|       0.00|11508632.89|       0.00|
|     8932|    8932|2020-11-01|       0.00|   17285502.00|       0.00|       0.00|       0.00|
|    12559|    2559|2020-11-01|       0.00|   12060563.97|       0.00|       0.00|       0.00|
|     5375|    5375|2020-11-01|       0.00|    7487815.21|       0.00| 7487815.21|       0.00|
|    16461|    6461|2020-11-01|16558734.00|          0.00|       0.00|       0.00|16558734.00|
|     1752|    1752|2020-11-01|       0.00|    9998839.69|       0.00| 9998839.69|       0.00|
|    18981|    8981|2020-11-01|17299614.00|          0.00|       0.00|       0.00|17299614.00|
|     6493|    6493|2020-11-01|11822645.61|       

In [51]:
//Построение витрины данных (corporate_payments)
val corporate_payments = clientsAndAccountsOpRate.groupBy($"AccDB".as("AccountID"), 
                                                          $"ClientDB".as("ClientId"), 
                                                          $"DateOp".as("CutoffDt"))
        .agg(round(sum(when($"AccDB" === $"AccountDB", $"AmountRub").otherwise(0)), 2).as("PaymentAmt"),
             round(sum(when($"AccDB" === $"AccountCR", $"AmountRub").otherwise(0)), 2).as("EnrollementAmt"),
             round(sum(when($"AccDB" === $"AccountDB" && $"NumCR".startsWith("40702"),$"AmountRub").otherwise(0)), 2).as("TaxAmt"),
             round(sum(when($"AccDB" === $"AccountCR" && $"NumDB".startsWith("40802"),$"AmountRub").otherwise(0)), 2).as("ClearAmt")
            )

corporate_payments = [AccountID: int, ClientId: int ... 5 more fields]


[AccountID: int, ClientId: int ... 5 more fields]

In [52]:
 corporate_payments.show()

+---------+--------+----------+-----------+--------------+-----------+-----------+
|AccountID|ClientId|  CutoffDt| PaymentAmt|EnrollementAmt|     TaxAmt|   ClearAmt|
+---------+--------+----------+-----------+--------------+-----------+-----------+
|     5167|    5167|2020-11-01|       0.00|   10413848.13|       0.00|       0.00|
|     5375|    5375|2020-11-01|       0.00|    7487815.21|       0.00| 7487815.21|
|      557|     557|2020-11-01| 7527546.61|          0.00| 7527546.61|       0.00|
|     5835|    5835|2020-11-01|       0.00|   11965278.00|       0.00|11965278.00|
|     1201|    1201|2020-11-01|10643746.77|          0.00|       0.00|       0.00|
|     6563|    6563|2020-11-01|       0.00|    7750042.45|       0.00| 7750042.45|
|     7163|    7163|2020-11-01|       0.00|    7882480.45|       0.00| 7882480.45|
|     2605|    2605|2020-11-01|10956515.85|          0.00|10956515.85|       0.00|
|     8471|    8471|2020-11-01|       0.00|    8171195.29|       0.00|       0.00|
|   

In [55]:
 // запишем витрину corporate_payments в parquet-файл
// parquet файл - экономит место, сохраняет данные с сжатом виде
val path = "data_parquets/"+pDateCommon+"/corporate_payments"
corporate_payments.write
    .format("parquet")
    .option("path", path)
    .mode("Overwrite").save()

path = data_parquets/2020-11-01/corporate_payments


data_parquets/2020-11-01/corporate_payments

In [56]:
//Построение витрины данных (corporate_account)
val corporate_account = corporate_payments
    .join(clientsAndAccounts.select($"AccountNum", 
                                    $"AccountID", 
                                    $"DateOpen", 
                                    $"ClientName"), "AccountID")
    .withColumn("TotalAmt", round($"PaymentAmt" + $"EnrollementAmt",2))
    .select($"AccountID", $"AccountNum", $"DateOpen", $"ClientId", $"ClientName", $"TotalAmt", $"CutoffDt")
    .orderBy("AccountID", "CutoffDt")

corporate_account.show() 

+---------+--------------------+----------+--------+----------+-----------+----------+
|AccountID|          AccountNum|  DateOpen|ClientId|ClientName|   TotalAmt|  CutoffDt|
+---------+--------------------+----------+--------+----------+-----------+----------+
|        1|40702810927050000337|2020-11-01|       1|  Клиент 1|10376422.77|2020-11-01|
|        4|40802810800000030701|2020-11-04|       4|  Клиент 4|14660670.00|2020-11-01|
|        5|40802810300000011071|2020-11-01|       5|  Клиент 5| 7405703.65|2020-11-01|
|        8|40802810409260005894|2020-11-04|       8|  Клиент 8|12605904.93|2020-11-01|
|        9|40802810905030000004|2020-11-01|       9|  Клиент 9|11721846.00|2020-11-01|
|       12|40702810836260000957|2020-11-04|      12| Клиент 12| 9614769.49|2020-11-01|
|       13|40702810000000150277|2020-11-01|      13| Клиент 13|10379096.01|2020-11-01|
|       16|40702810400000099734|2020-11-04|      16| Клиент 16|14664198.00|2020-11-01|
|       17|40702810600000057437|2020-11-01|

[AccountID: int, AccountNum: string ... 5 more fields]

corporate_account = [AccountID: int, AccountNum: string ... 5 more fields]


In [57]:
 // запишем витрину corporate_payments в parquet-файл
// parquet файл - экономит место, сохраняет данные с сжатом виде
val path = "data_parquets/"+pDateCommon+"/corporate_account"
corporate_account.write
    .format("parquet")
    .option("path", path)
    .mode("Overwrite").save()

path = data_parquets/2020-11-01/corporate_account


data_parquets/2020-11-01/corporate_account

In [58]:
//Построение витрины данных (corporate_info)
val corporate_info = corporate_account
    .join(clientsAndAccounts.select($"ClientId".as("Clid"),
                                    $"ClientName".as("ClName"),
                                    $"Type",
                                    $"Form",
                                    $"RegisterDate"), $"Clid"===$"ClientId","left")



corporate_info.select($"ClientId",
                      $"ClientName",
                      $"Type",
                      $"Form",
                      $"RegisterDate",
                      $"TotalAmt",
                      $"CutoffDt").show()


+--------+----------+----+----+------------+-----------+----------+
|ClientId|ClientName|Type|Form|RegisterDate|   TotalAmt|  CutoffDt|
+--------+----------+----+----+------------+-----------+----------+
|       1|  Клиент 1|   Ф|   -|  2020-11-01|10376422.77|2020-11-01|
|       1|  Клиент 1|   Ф|   -|  2020-11-01|10376422.77|2020-11-01|
|       4|  Клиент 4|   Ю|  ИП|  2020-11-04|14660670.00|2020-11-01|
|       4|  Клиент 4|   Ю|  ИП|  2020-11-04|14660670.00|2020-11-01|
|       5|  Клиент 5|   Ф|   -|  2020-11-01| 7405703.65|2020-11-01|
|       5|  Клиент 5|   Ф|   -|  2020-11-01| 7405703.65|2020-11-01|
|       8|  Клиент 8|   Ю| ПАО|  2020-11-01|12605904.93|2020-11-01|
|       8|  Клиент 8|   Ю| ПАО|  2020-11-01|12605904.93|2020-11-01|
|       9|  Клиент 9|   Ф|   -|  2020-11-02|11721846.00|2020-11-01|
|       9|  Клиент 9|   Ф|   -|  2020-11-02|11721846.00|2020-11-01|
|      12| Клиент 12|   Ю|  ИП|  2020-11-01| 9614769.49|2020-11-01|
|      12| Клиент 12|   Ю|  ИП|  2020-11-01| 961

corporate_info = [AccountID: int, AccountNum: string ... 10 more fields]


[AccountID: int, AccountNum: string ... 10 more fields]

In [59]:
 // запишем витрину corporate_info в parquet-файл
// parquet файл - экономит место, сохраняет данные с сжатом виде
val path = "data_parquets/"+pDateCommon+"/corporate_info"
corporate_info.write
    .format("parquet")
    .option("path",path)
    .mode("Overwrite").save()

path = data_parquets/2020-11-01/corporate_info


data_parquets/2020-11-01/corporate_info